In [46]:
import os, copy
import torch
import numpy as np
import nibabel as nib
import glob 
import pandas as pd
from monai.transforms import LoadImage
from monai.metrics import DiceMetric

In [111]:
val_results_path = "val_results"
gt_path = "labels_tumor_cropped"

In [112]:
val_results = glob.glob(f'{val_results_path}/**.nii.gz', recursive=False)

In [113]:
liver_ids = [val_results[idx].split('/')[1].split('_pred.nii.gz')[0] for idx in range(len(val_results))] 
gt_paths = [f'{gt_path}/{liver_ids[idx]}.nii.gz' for idx in range(len(liver_ids))]

In [114]:
data = {
    'liver_id' : liver_ids,
    'gt_paths': gt_paths,
    'val_paths': val_results   
       }



In [115]:
df = pd.DataFrame(data)

In [79]:
example_gt = "labels_tumor_cropped/liver_94.nii.gz"	
example_val = "val_results/liver_94_pred.nii.gz"

In [60]:
seg_gt = nib.load(example_gt)
seg_val = nib.load(example_val)

seg_gt_data = seg_gt.get_fdata()
seg_val_data = seg_val.get_fdata()

In [76]:
min_shape = tuple(min(s1, s2) for s1, s2 in zip(seg_gt_data.shape, seg_val_data.shape)) 
slices = tuple(slice(0, s) for s in min_shape) 

gt_cropped = seg_gt_data[slices] 
val_cropped = seg_val_data[slices]

In [140]:
from scipy.ndimage import zoom 
def compute_dice(row, epsilon=1e-8):

    gt_path = row['gt_paths']
    pred_path = row['val_paths']
    
    try:
        gt_data = nib.load(gt_path).get_fdata()
        pred_data = nib.load(pred_path).get_fdata()
        
        # resizing prediction to GT shape by padding
        if pred_data.shape != gt_data.shape:
            zoom_factors = [t / s for s, t in zip(pred_data.shape, gt_data.shape)]
            pred_data = zoom(pred_data, zoom_factors, order=0) 
        
        
        pred_bin = (pred_data > 0).astype(bool)
        gt_bin = (gt_data > 0).astype(bool)
        
        # DICE
        intersection = np.logical_and(pred_bin, gt_bin).sum()
        total = pred_bin.sum() + gt_bin.sum()
        
        if total == 0:
            return 1.0  
        
        dice = (2.0 * intersection + epsilon) / (total + epsilon)
        return dice
        
    except Exception as e:
        print(f"Error processing GT: {gt_path}, Pred: {pred_path}. Error: {e}")
        return np.nan





In [141]:
df

,liver_id,gt_paths,val_paths,dice_score
0,liver_94,labels_tumor_cropped/liver_94.nii.gz,val_results/liver_94_pred.nii.gz,6.572133e-01
1,liver_69,labels_tumor_cropped/liver_69.nii.gz,val_results/liver_69_pred.nii.gz,6.795252e-01
2,liver_64,labels_tumor_cropped/liver_64.nii.gz,val_results/liver_64_pred.nii.gz,9.471631e-01
3,liver_4,labels_tumor_cropped/liver_4.nii.gz,val_results/liver_4_pred.nii.gz,8.410075e-01
4,liver_98,labels_tumor_cropped/liver_98.nii.gz,val_results/liver_98_pred.nii.gz,8.893028e-01
5,liver_123,labels_tumor_cropped/liver_123.nii.gz,val_results/liver_123_pred.nii.gz,8.071070e-01
6,liver_48,labels_tumor_cropped/liver_48.nii.gz,val_results/liver_48_pred.nii.gz,8.608635e-01
7,liver_108,labels_tumor_cropped/liver_108.nii.gz,val_results/liver_108_pred.nii.gz,7.360531e-01
8,liver_102,labels_tumor_cropped/liver_102.nii.gz,val_results/liver_102_pred.nii.gz,6.349069e-01
9,liver_57,labels_tumor_cropped/liver_57.nii.gz,val_results/liver_57_pred.nii.gz,6.264591e-01


In [142]:
df['dice_score'] = df.apply(compute_dice, axis = 1)

In [123]:
df.to_csv('dice_scores_val.csv')